In [175]:
import os
from copy import deepcopy

import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split,KFold,GridSearchCV,cross_val_score

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import SplineTransformer,PolynomialFeatures,StandardScaler,MinMaxScaler



from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

# MODELS
from sklearn.neighbors import NearestNeighbors

In [176]:
def mean(list):
    return sum(list)/len(list)

def is_outlier(points, thresh=3.5):
    """
    Returns a boolean array with True if points are outliers and False 
    otherwise.

    Parameters:
    -----------
        points : An numobservations by numdimensions array of observations
        thresh : The modified z-score to use as a threshold. Observations with
            a modified z-score (based on the median absolute deviation) greater
            than this value will be classified as outliers.

    Returns:
    --------
        mask : A numobservations-length boolean array.

    References:
    ----------
        Boris Iglewicz and David Hoaglin (1993), "Volume 16: How to Detect and
        Handle Outliers", The ASQC Basic References in Quality Control:
        Statistical Techniques, Edward F. Mykytka, Ph.D., Editor. 
    """
    if len(points.shape) == 1:
        points = points[:,None]
    median = np.median(points, axis=0)
    diff = np.sum((points - median)**2, axis=-1)
    diff = np.sqrt(diff)
    med_abs_deviation = np.median(diff)

    modified_z_score = 0.6745 * diff / med_abs_deviation

    return modified_z_score > thresh

In [2]:
# COLLABORATIVE RECS -- Future application; need users

# Python --> Crab library

In [3]:
# User Input Considerations:

# I'm Feeling Lucky -> Drop a random dot in the housing map

## Price Range: Min - Max

## Property Type

# Minimum Requirements ^^

## State

## City

## Zip Code

## Property Type

## No. of Bedrooms

## No. of Bathrooms

## Square Footage

## Year Built



### NEED TO IDENTIFY WHEN DATA IS SPARSE FOR A CERTAIN AREA AND PRESENT APOLOGY MESSAGE


In [4]:
## CONSIDERATIONS

# INTRINSICALLY ASSIGN EACH PROPERTY PREDICTOR ITS LOCATION

# INTRINSICALLY ASSIGN EACH PROPERTY PREDICTOR ITS PROPERTY TYPE

# RANK THE CHOSEN NEIGHBORS IN A QUEUE, FILTER THE QUEUE BY PRICE RANGE, AND RETURN THE TOP 1-15

In [5]:
aws_bucket=os.getenv('AWS_RE_BUCKET')
aws_key=os.getenv('AWS_RE_BUCKET_KEY')
aws_secret=os.getenv('AWS_RE_BUCKET_SECRET')

date="2023-06-04"
state="CA"
states=['AL',
 'NE',
 'AK',
 'NV',
 'AZ',
 'NH',
 'AR',
 'NJ',
 'CA',
 'NM',
 'CO',
 'NY',
 'CT',
 'NC',
 'DE',
 'ND',
 'DC',
 'OH',
 'FL',
 'OK',
 'GA',
 'OR',
 'HI',
 'PA',
 'ID',
 'PR',
 'IL',
 'RI',
 'IN',
 'SC',
 'IA',
 'SD',
 'KS',
 'TN',
 'KY',
 'TX',
 'LA',
 'UT',
 'ME',
 'VT',
 'MD',
 'VA',
 'MA',
 'VI',
 'MI',
 'WA',
 'MN',
 'WV',
 'MS',
 'WI',
 'MO',
 'WY',
 'MT',
]

In [16]:
# PULLING IN DATA

df_list=[]
for st in states:
    try:
        df = pd.read_parquet(path=aws_bucket+f"/{date}/{st}.parquet",storage_options={"key":aws_key,"secret":aws_secret})
        df_list.append(df)
    except:
        pass
    
df=pd.concat(df_list)

In [17]:
df['PROPERTY TYPE'].unique()

array(['Single Family Residential', 'Vacant Land',
       'Mobile/Manufactured Home', 'Townhouse', 'Ranch',
       'Multi-Family (5+ Unit)', 'Multi-Family (2-4 Unit)', 'Condo/Co-op',
       'Parking', 'Other', 'Timeshare', 'Moorage', 'Unknown', 'None'],
      dtype=object)

In [18]:
df_copy=deepcopy(df)

In [161]:
df=df_copy

In [163]:
# PREPROCESSING
prop_types=['Single Family Residential', 'Mobile/Manufactured Home','Townhouse', 'Multi-Family (2-4 Unit)', 'Condo/Co-op','Multi-Family (5+ Unit)']

df=df.applymap(lambda x: np.nan if x=='nan' else x)

zip_count = df[['ZIP OR POSTAL CODE']].nunique().iloc[0]

df=df[(df['CITY']=='Houston')&(df['STATE OR PROVINCE']=='TX')] # TEMP LIMITER 1

df=df[df['PROPERTY TYPE']=='Single Family Residential'] # TEMP LIMITER 2

df=df[['ADDRESS','CITY','STATE OR PROVINCE','ZIP OR POSTAL CODE','PROPERTY TYPE','PRICE','BEDS','BATHS','SQUARE FEET','YEAR BUILT','DAYS ON MARKET','LATITUDE','LONGITUDE']].dropna(axis=0).reset_index(drop=True).apply(lambda row: pd.to_numeric(row,errors='ignore'))

df=df[df['PROPERTY TYPE'].isin(prop_types)]

#temp=pd.get_dummies(data=df['PROPERTY TYPE'])

#df_full=pd.concat([df,temp],axis=1)

df_full=df

df_final=df_full.drop(['ADDRESS','CITY','STATE OR PROVINCE','ZIP OR POSTAL CODE','PROPERTY TYPE'],axis=1)


In [164]:
df_full

,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PROPERTY TYPE,PRICE,BEDS,BATHS,SQUARE FEET,YEAR BUILT,DAYS ON MARKET,LATITUDE,LONGITUDE
0,741 Winding Rd,Houston,TX,77504,Single Family Residential,427500.0,3.0,3.0,2021.0,1930.0,102.0,29.635577,-95.200023
1,10422 Lyndon Meadows Dr,Houston,TX,77095,Single Family Residential,320000.0,3.0,2.5,1609.0,2004.0,17.0,29.925922,-95.667346
2,7411 Dew Mist Ln,Houston,TX,77095,Single Family Residential,286000.0,3.0,2.0,2022.0,1984.0,1.0,29.885824,-95.657289
3,17014 Catskill Bluff Ln,Houston,TX,77095,Single Family Residential,375000.0,5.0,3.5,3894.0,2004.0,1.0,29.921531,-95.674598
4,7706 View Park Ln,Houston,TX,77095,Single Family Residential,274500.0,3.0,2.0,1375.0,1987.0,2.0,29.891378,-95.656951
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4710,10055 Sharpton Dr,Houston,TX,77038,Single Family Residential,175000.0,4.0,2.5,2098.0,2006.0,83.0,29.911716,-95.438544
4711,10053 Sharpton Dr,Houston,TX,77038,Single Family Residential,175000.0,4.0,2.5,2098.0,2006.0,83.0,29.911655,-95.438529
4712,2743 Oriole Wood Ct,Houston,TX,77038,Single Family Residential,320000.0,5.0,3.5,2552.0,2011.0,93.0,29.906036,-95.459280
4713,2507 Manacor Cir,Houston,TX,77038,Single Family Residential,250000.0,3.0,2.0,2515.0,1981.0,100.0,29.899882,-95.447457


In [169]:
df_NN=df_final[['BEDS','BATHS','SQUARE FEET','YEAR BUILT']]

In [170]:
df_NN.reset_index(drop=True)

,BEDS,BATHS,SQUARE FEET,YEAR BUILT
0,3.0,3.0,2021.0,1930.0
1,3.0,2.5,1609.0,2004.0
2,3.0,2.0,2022.0,1984.0
3,5.0,3.5,3894.0,2004.0
4,3.0,2.0,1375.0,1987.0
...,...,...,...,...
4710,4.0,2.5,2098.0,2006.0
4711,4.0,2.5,2098.0,2006.0
4712,5.0,3.5,2552.0,2011.0
4713,3.0,2.0,2515.0,1981.0


In [171]:
# Define the transformations for each column
preprocessor = ColumnTransformer(
transformers=[
    ('standard_scaler',MinMaxScaler(),['BEDS','BATHS','SQUARE FEET','YEAR BUILT']),
])

# Apply the transformations in a pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

df_NN=pipeline.fit_transform(df_NN)

In [172]:
input_df = pd.DataFrame({'BEDS':[2],'BATHS':[1.5],'SQUARE FEET':[1209],'YEAR BUILT':[2017]})
input_df

pipeline.fit_transform(input_df)



array([[0., 0., 0., 0.]])

# NEED TO FIGURE OUT HOW TO SCALE INPUT DATA WITH SCALE FOR EXISTING DATA

# OR PERFORM WEIGHTS TESTING TO MINIMIZE IMPACT OF SQUARE FEET

In [173]:

knn=NearestNeighbors(n_neighbors=100)
knn.fit(df_NN)

distances, indices = knn.kneighbors(pipeline.fit_transform(input_df))


# CURRENT IMPLEMENTATION PUTS TOO MUCH FOCUS ON EXACT

# E.G., For 2BD, 1.5BA, 1300SQF would choose 4BD,3BA, 1300SQF over 2BD, 1.5BA, 1250SQF
        
        
    
    

In [174]:
df_full.iloc[indices[0][0]]

ADDRESS                        1310 Beachton St
CITY                                    Houston
STATE OR PROVINCE                            TX
ZIP OR POSTAL CODE                        77007
PROPERTY TYPE         Single Family Residential
PRICE                                  425000.0
BEDS                                        1.0
BATHS                                       1.0
SQUARE FEET                               752.0
YEAR BUILT                               1890.0
DAYS ON MARKET                              4.0
LATITUDE                              29.770322
LONGITUDE                             -95.36768
Name: 2466, dtype: object